Create Dataset folders

In [1]:
#Importing the os module, which provides functions for interacting with the operating system
import os

# Create two folders: 'Brain Tumor' and 'Healthy' inside '/content'
tumor_dir = '/content/Brain Tumor'
healthy_dir = '/content/Healthy'

# Check if the folders already exist and create them if they don't
if not os.path.exists(tumor_dir):
    os.makedirs(tumor_dir)

if not os.path.exists(healthy_dir):
    os.makedirs(healthy_dir)

# Verify the folders are created
print("Created folders:")
print("Brain Tumor folder:", os.path.exists(tumor_dir))
print("Healthy folder:", os.path.exists(healthy_dir))


Created folders:
Brain Tumor folder: True
Healthy folder: True


Import Dependencies

In [2]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import matplotlib.pyplot as plt
from PIL import Image

Load the images

In [3]:
# Load the metadata CSV
metadata = pd.read_csv('/content/metadata.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/metadata.csv'

In [ ]:
# Display the first few rows of metadata
metadata.head()

In [ ]:
# Initialize lists to store image data and labels
images = []
labels = []

In [ ]:
# Define the base directory where the images are located
base_dir = '/content'

In [ ]:
# Define the paths for 'tumor' and 'normal' images
tumor_dir = os.path.join(base_dir, 'Brain Tumor')
normal_dir = os.path.join(base_dir, 'Healthy')

In [ ]:
# Iterate through the metadata and load the images
for _, row in metadata.iterrows():
    # Determine the image path and label
    if row['class'] == 'tumor':
        image_path = os.path.join(tumor_dir, row['image'])
        label = 1
    else:  # 'normal'
        image_path = os.path.join(normal_dir, row['image'])
        label = 0

In [ ]:
# Check if image exists
    if os.path.exists(image_path):
        # Load the image and ensure it's in RGB mode
        img = Image.open(image_path).convert('RGB')

        # Resize the image to a fixed size (150x150)
        img = img.resize((150, 150))

        # Convert the image to a numpy array and normalize
        img_array = img_to_array(img) / 255.0
        images.append(img_array)

        # Append the label
        labels.append(label)
    else:
        print(f"Image {image_path} not found")

In [ ]:
# Convert the images and labels to numpy arrays
images = np.array(images)
labels = np.array(labels)

Training the model

In [ ]:
# Split the dataset into training and validation sets (80-20 split)
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

In [ ]:
# Build a simple CNN model for brain tumor detection
model = models.Sequential()

# Add convolutional layers with pooling
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Flatten the output and add dense layers
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))  # Binary classification (tumor or not)

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

Save the model

In [ ]:
# Save the trained model
model.save('/content/brain_tumor_model.h5')

Plot accuracy

In [ ]:
# Plot training & validation accuracy
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.show()